

#Question 3 (50 Marks)

This question is about Named Entity Recognition

In [4]:
### do not change the code in this cell
# make sure you run this cell
import nltk

# This is a list of sentences with NER tags which we will use in this question.
tagged_sents=[["tim_PER", "cook_PER", "is", "the", "current", "ceo", "of", "apple_ORG", "inc._ORG", ",", "headquartered", "in", "cupertino_LOC", "."],
              ["fiona_PER", "apple_PER", "was", "born", "in", "new_LOC", "york_LOC", "and", "her", "debut", "album", "was", "released", "by", "columbia_ORG", "records_ORG", "."],
              ["the_LOC", "big_LOC", "apple_LOC", "is", "a", "nickname", "for", "new_LOC", "york_LOC", "city_LOC", "."]]


a) Follow the steps below to preprocess the NER tagged sentences, producing a list of lists of tokens and a list of lists of NER tags.

i) Use the `.split()` method to separate the tokens from the NER tags in the list `tagged_sents`, and strip off the tags to produce a list of lists of strings, called `tokens`, corresponding to just the tokens in each sentence.

So, for example `[["Alice_PER", "runs"], ["Bob_PER", "walks"]]` would produce `[["Alice", "runs"], ["Bob", "walks"]]`.

(8 marks)

In [5]:
# Initialize the list to hold lists of tokens
tokens = []

# Iterate over each sentence in tagged_sents
for sent in tagged_sents:
    # Use a list comprehension to split each tagged token and take the first part (the token itself)
    sent_tokens = [token.split('_')[0] for token in sent]
    # Append the list of tokens for this sentence to the tokens list
    tokens.append(sent_tokens)

# Print the tokens to verify the output
print("Tokens:")
for sent_tokens in tokens:
    print(sent_tokens)

Tokens:
['tim', 'cook', 'is', 'the', 'current', 'ceo', 'of', 'apple', 'inc.', ',', 'headquartered', 'in', 'cupertino', '.']
['fiona', 'apple', 'was', 'born', 'in', 'new', 'york', 'and', 'her', 'debut', 'album', 'was', 'released', 'by', 'columbia', 'records', '.']
['the', 'big', 'apple', 'is', 'a', 'nickname', 'for', 'new', 'york', 'city', '.']


ii) Use the `.split()` method to separate the tokens from the NER tags in the list `tagged_sents`, producing a list of lists of NER tags, called `tags`. Tokens without an NER tag should be tagged with the letter `"O"`.

So, for example `[["Alice_PER", "runs"], ["chase", "Bob_PER"]]` would produce `[["PER", "O"], ["O", "PER"]]`.

(6 marks)

In [6]:
# Initialize the list to hold lists of NER tags
tags = []

# Iterate over each sentence in tagged_sents
for sent in tagged_sents:
    # Use a list comprehension to split each tagged token and take the tag part, default to "O"
    sent_tags = [token.split('_')[1] if '_' in token else 'O' for token in sent]
    # Append the list of tags for this sentence to the tags list
    tags.append(sent_tags)

# Print the tags to verify the output
print("NER Tags:")
for sent_tags in tags:
    print(sent_tags)

NER Tags:
['PER', 'PER', 'O', 'O', 'O', 'O', 'O', 'ORG', 'ORG', 'O', 'O', 'O', 'LOC', 'O']
['PER', 'PER', 'O', 'O', 'O', 'LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'ORG', 'ORG', 'O']
['LOC', 'LOC', 'LOC', 'O', 'O', 'O', 'O', 'LOC', 'LOC', 'LOC', 'O']


b) Now, follow the steps below to derive the transition and emission probabilities of a Hidden Markov Model for the NER tag sequences.

i) Describe the two assumptions that a Hidden Markov Model for sequence tagging is based on.

(4 marks)

A Hidden Markov Model (HMM) for sequence tagging is based on two key assumptions:
<ol>
    <li><b>Markov Assumption (or First-Order Markov Property)</b>: This assumption states that the probability of a particular state (or tag in the context of sequence tagging) depends only on the previous state. In other words, the future state is conditionally independent of the past states given the present state. For sequence tagging, this means that the probability of a tag at position $i$ in a sequence depends only on the tag at position $i-1$ and not on tags before $i-1$.</li>
    <li><b>Output Independence (or Emission Independence)</b>: This assumption posits that the probability of an observed symbol (or token in the context of sequence tagging) being emitted by a state (or tag) is independent of the other observed symbols and the sequence of states that led up to the current state. In sequence tagging, this translates to the probability of a word being associated with a tag being independent of surrounding words and their respective tags. The observed word (emission) is only dependent on the current state (tag) that emits it.</li>
</ol>

These two assumptions allow HMMs to model the joint probability of a sequence of observed symbols (words in a sentence) and a sequence of states (tags) in a computationally tractable way. However, it's important to note that real-world sequences often have dependencies that go beyond these assumptions, which can limit the accuracy of HMMs. Despite this, HMMs can still perform quite well in many sequence tagging tasks like part-of-speech tagging, speech recognition, and, as in this case, named entity recognition (NER).

ii) Create a dictionary, called `nercounts`, containing the total counts for each tag in the list `tags`. The keys of `nercounts` should be tags and the values should be the frequencies of those tags.

(4 marks)

In [7]:
from collections import defaultdict

# Initialize defaultdict to hold tag counts (default to 0 for any key that does not exist)
nercounts = defaultdict(int)

# Iterate over each list of tags in the tags list
for sent_tags in tags:
    # Count the occurrences of each tag
    for tag in sent_tags:
        nercounts[tag] += 1

# Convert defaultdict to a regular dictionary for the final output
nercounts = dict(nercounts)

# Print the nercounts to verify the output
print("NER Counts:")
print(nercounts)

NER Counts:
{'PER': 4, 'O': 25, 'ORG': 4, 'LOC': 9}


iii) Using the lists `tags` and `tokens`, calculate the emission probabilities: $$p(token|tag)$$

Put these probabilities in a dictionary of dictionaries, called `emission`, with outer keys being NER tags, inner keys being tokens and inner values being the emission probabilities.

(8 marks)

In [8]:
# Initialize nested defaultdict to hold emission counts
# The outer dict holds tags, and the inner dict holds tokens and their counts
emission_counts = defaultdict(lambda: defaultdict(int))

# Iterate over each sentence's tags and corresponding tokens
for sent_tags, sent_tokens in zip(tags, tokens):
    for tag, token in zip(sent_tags, sent_tokens):
        # Count the occurrences of the token for the given tag
        emission_counts[tag][token] += 1

# Initialize the emission probability dictionary
emission = defaultdict(dict)

# Calculate the emission probabilities
for tag, token_counts in emission_counts.items():
    # Calculate the total count of the tag
    total_tag_count = nercounts[tag]
    # Calculate the emission probability for each token under the tag
    for token, count in token_counts.items():
        emission[tag][token] = count / total_tag_count

# Convert defaultdict to a regular dictionary for the final output
emission = {tag: dict(token_probs) for tag, token_probs in emission.items()}

# Print the emission probabilities to verify the output
print("Emission Probabilities:")
for tag, token_probs in emission.items():
    print(f"{tag}: {token_probs}")

Emission Probabilities:
PER: {'tim': 0.25, 'cook': 0.25, 'fiona': 0.25, 'apple': 0.25}
O: {'is': 0.08, 'the': 0.04, 'current': 0.04, 'ceo': 0.04, 'of': 0.04, ',': 0.04, 'headquartered': 0.04, 'in': 0.08, '.': 0.12, 'was': 0.08, 'born': 0.04, 'and': 0.04, 'her': 0.04, 'debut': 0.04, 'album': 0.04, 'released': 0.04, 'by': 0.04, 'a': 0.04, 'nickname': 0.04, 'for': 0.04}
ORG: {'apple': 0.25, 'inc.': 0.25, 'columbia': 0.25, 'records': 0.25}
LOC: {'cupertino': 0.1111111111111111, 'new': 0.2222222222222222, 'york': 0.2222222222222222, 'the': 0.1111111111111111, 'big': 0.1111111111111111, 'apple': 0.1111111111111111, 'city': 0.1111111111111111}


iv) Using the list `tags`, calculate the transition probabilities: $$p(tag_i|tag_{i-1})$$

You will need to keep track of both the current tag, $tag_i$, and the previous tag, $tag_{i-1}$, and you will need to introduce a special tag, e.g. "START", for the previous tag at the beginning of a sequence.

For example, the sequence of tags `["PER", "O"]` would give rise to the following values of $i$, $tag_{i-1}$ and $tag_i$:

|i|$tag_{i-1}$|$tag_i$|
|---|---|---|
|0|START|PER|
|1|PER|O|

Calculate the conditional probabilities of $tag_i$ given $tag_{i-1}$ and put the results in a dictionary of dictionaries, called `transition`, with outer keys being previous tags, inner keys being current tags and inner values being transition probabilities.

(10 marks)

In [9]:
# Initialize a defaultdict for transition counts
transition_counts = defaultdict(lambda: defaultdict(int))

# Initialize counts for the special "START" tag
start_tag_counts = defaultdict(int)

# Iterate over each list of tags in the tags list
for sent_tags in tags:
    # Insert "START" at the beginning of each tag sequence
    sent_tags = ["START"] + sent_tags
    
    # Count the transitions from each tag to the next tag
    for i in range(len(sent_tags) - 1):
        prev_tag = sent_tags[i]
        curr_tag = sent_tags[i+1]
        transition_counts[prev_tag][curr_tag] += 1
        
        # Increment the start tag count each time a sentence begins
        if prev_tag == "START":
            start_tag_counts[prev_tag] += 1

# Initialize the transition probability dictionary
transition = defaultdict(dict)

# Calculate the transition probabilities
for prev_tag, curr_tag_counts in transition_counts.items():
    # Calculate the total count of the previous tag
    total_prev_tag_count = sum(curr_tag_counts.values())
    if prev_tag == "START":
        total_prev_tag_count = start_tag_counts[prev_tag]
    
    # Calculate the transition probability for each current tag given the previous tag
    for curr_tag, count in curr_tag_counts.items():
        transition[prev_tag][curr_tag] = count / total_prev_tag_count

# Convert defaultdict to a regular dictionary for the final output
transition = {prev_tag: dict(curr_tag_probs) for prev_tag, curr_tag_probs in transition.items()}

# Print the transition probabilities to verify the output
print("Transition Probabilities:")
for prev_tag, curr_tag_probs in transition.items():
    print(f"{prev_tag}: {curr_tag_probs}")

Transition Probabilities:
START: {'PER': 0.6666666666666666, 'LOC': 0.3333333333333333}
PER: {'PER': 0.5, 'O': 0.5}
O: {'O': 0.7727272727272727, 'ORG': 0.09090909090909091, 'LOC': 0.13636363636363635}
ORG: {'ORG': 0.5, 'O': 0.5}
LOC: {'O': 0.4444444444444444, 'LOC': 0.5555555555555556}


v) Given a sequence of tokens, we want to find the most probable sequence of tags corresponding to those tokens. The brute force approach would simply evaluate the probability of every sequence of tags. Explain why we would want to avoid that approach and describe an alternative algorithm.

(6 marks)

The brute force approach to finding the most probable sequence of tags for a given sequence of tokens involves evaluating the probability of every possible sequence of tags, which is computationally infeasible for several reasons:
<ol>
    <li><b>Combinatorial Explosion</b>: For a sequence of tokens of length $n$` and a set of possible tags of size $t$, there are $t^n$ possible sequences of tags. For even moderately sized $n$ and $t$, this number becomes astronomically large, leading to a combinatorial explosion of possibilities.</li>
    <li><b>Inefficiency</b>: Evaluating every possible sequence is highly inefficient because the vast majority of tag sequences are extremely unlikely and do not need to be considered. Brute force does not leverage the structure of the problem or any heuristics that could drastically reduce the search space.
        <li><b>Time Complexity</b>: The time complexity of evaluating all possible sequences is exponential, making it impractical for real-time or even offline processing of sentences, especially when dealing with large corpora.</li>
</ol>

An alternative algorithm to the brute force approach is the <b>Viterbi algorithm</b>, which efficiently finds the most probable sequence of tags using dynamic programming. The Viterbi algorithm works as follows:
<ol>
    <li><b>Initialization</b>: Start with the probabilities of the first token being generated by each tag and multiply by the probability of each tag being at the start of a sentence (the "<b>START</b>" transition probabilities).</li>
    <li><b>Recursion</b>: For each subsequent token, calculate the probabilities of each tag by considering the transition probabilities from all previous tags to the current tag and the emission probability of the current token given the current tag. Keep track of the most probable path to each tag.</li>
    <li><b>Termination</b>: After processing all tokens, find the end state with the highest probability.</li>
    <li><b>Path Backtracking</b>: Trace back through the saved paths to reconstruct the most probable sequence of tags from the end state to the start state.</li>

The Viterbi algorithm exploits the Markov property of the HMM, which states that the probability of a tag at a given position depends only on the previous tag and not on the entire sequence of previous tags. By storing only the highest probabilities and paths at each step, the Viterbi algorithm avoids redundant calculations and reduces the problem to polynomial time complexity, specifically $O(n*t^2)$, which is much more manageable than the exponential complexity of the brute force approach.

vi) If we are tagging a sequence and the current token is `"apple"` and the previous tag is `"LOC"`, then we want know to which tag in the current position  would maximise the probability of seeing that token.

Define a function that takes a token, a previous tag, a dictionary of emission probabilities and a dictionary of transition probabilities and returns a dictionary of the probabilities $p(token, tag_i | tag_{i-1})$ for all values of $tag_i$.

Apply this function to the case where the current token is `"apple"` and the previous tag is `"LOC"`.

(4 marks)

In [10]:
def max_probability_token(token, prev_tag, emission_probs, transition_probs):
    # Initialize a dictionary to hold the probabilities for the current token and all possible current tags
    probabilities = {}
    
    # Iterate over all possible current tags
    for curr_tag in emission_probs.keys():
        # Calculate the transition probability for the current tag given the previous tag
        trans_prob = transition_probs[prev_tag].get(curr_tag, 0)
        
        # Calculate the emission probability for the current token given the current tag
        emit_prob = emission_probs[curr_tag].get(token, 0)
        
        # Calculate the combined probability of the current token and current tag given the previous tag
        probabilities[curr_tag] = trans_prob * emit_prob
    
    return probabilities

# Apply the function to the token "apple" and the previous tag "LOC"
current_token = "apple"
previous_tag = "LOC"
probabilities = max_probability_token(current_token, previous_tag, emission, transition)

# Print the probabilities to verify the output
print(f"Probabilities for the token '{current_token}' given the previous tag '{previous_tag}':")
print(probabilities)

Probabilities for the token 'apple' given the previous tag 'LOC':
{'PER': 0.0, 'O': 0.0, 'ORG': 0.0, 'LOC': 0.06172839506172839}
